In [2]:
#Import modules
import urllib
import time
import datetime
import pandas as pd
import shutil
import os

In [7]:
#NOTE: the urls are iterated in reverse order from most recent to least

#Set Parliament and associated Session
#ex. 421 = 42nd Parliament, 1st session
parliament_session = ["412"]

#Set the issue number of the Senate session (must be corresponded with the correct date)
#2018-09-18 = Issue 226
issue_number = 162

#Get list of business dates (in descending order)
#Recommend moving in sections from end of June to the beginning of September of previous year, depending on session
datelist = pd.bdate_range(start="2014-09-16",end="2015-06-30").date.tolist()
datelist = datelist[::-1]

url_error = "Sorry, the content is not available."

#Simple function to get the url from the above information
def url_funct(session,issue_number,i):   
    #Convert 2 or 1 digit numbers to 3 digit (i.e. 99 to 099) to satisfy url requirements
    if len(str(issue_number)) == 2:
        issue_number_str = "0"+str(issue_number)
        url = "https://sencanada.ca/en/content/sen/chamber/"+session+"/debates/"+issue_number_str+"db_"+str(datelist[i])+"-e"
    elif len(str(issue_number)) == 1:
        issue_number_str = "00"+str(issue_number)
        url = "https://sencanada.ca/en/content/sen/chamber/"+session+"/debates/"+issue_number_str+"db_"+str(datelist[i])+"-e"
    else:
        url = "https://sencanada.ca/en/content/sen/chamber/"+session+"/debates/"+str(issue_number)+"db_"+str(datelist[i])+"-e"
    return url

for session in parliament_session:
    for num,i in enumerate(range(len(datelist))):
        starttime = time.time()
        print(url_funct(session,issue_number,i))
        html = urllib.request.urlopen(url_funct(session,issue_number,i))
    
        #Copy HTML of site onto local file
        #os.mkdir("/Senate Hansard Parliament "+session[::2]+" Session "+session[:1:-1])
        #file = open("/Senate Hansard Parliament "+session[::2]+" Session "+session[:1:-1]+"/"+session+"_"+str(issue_number)+"_"+str(datelist[i])+".html","wb")
        file = open("/"+session+"_"+str(issue_number)+"_"+str(datelist[i])+".html","wb")
        shutil.copyfileobj(html,file)
        file.close()
        
        #Check if HTML is invalid, delete file if true (also makes it so that issue_number only decreases if url was valid)
        #Currently convoluted, but only method I found thus far
        with open(session+"_"+str(issue_number)+"_"+str(datelist[i])+".html") as error_check_HTML:
            for line in error_check_HTML:
                if url_error in line:
                    print("The previous URL does not exist.")
                    os.remove(session+"_"+str(issue_number)+"_"+str(datelist[i])+".html")
                    issue_number += 1
                    break
                else:
                    pass
            issue_number -= 1
            
        #Code sleeps for 3 seconds after each iteration to avoid overcrowding site
        time.sleep(3.0 - ((time.time() - starttime) % 3.0))
        
        #Code sleeps for 30 seconds after every 30 iterations for additional delay
        if num == 30:
            time.sleep(30.0 - ((time.time() - starttime) % 30.0))

https://sencanada.ca/en/content/sen/chamber/412/debates/162db_2015-06-30-e


PermissionError: [Errno 13] Permission denied: '/Senate Hansard Parliament 42 Session 2'